###  Created by Luis Alejandro (alejand@umich.edu)
A single forward and backward pass for a softmax regressor using graphs. Compares results to analytical computations.

In [1]:
import numpy as np
import numpy.random as rnd

import sys
sys.path.append('../')

from utils.graphs.core import Param
from utils.graphs.core import DataHolder
from utils.graphs.core import Graph
from utils.graphs.core import Operation

from utils.graphs.nodes import linear_node
from utils.graphs.nodes import bias_node
from utils.graphs.nodes import softmax_node
from utils.graphs.nodes import mce_node
from utils.graphs.nodes import softmax_mce_node

from sklearn import datasets

In [2]:
dataset = datasets.load_iris()
predictors = dataset['data']
responses = dataset['target'].reshape(-1,1)
m,d = predictors.shape
n = len(np.unique(responses))

In [3]:
# rnd.seed(1)
X_node = DataHolder()
y_node = DataHolder()
w_node = Param((d,n))
b_node = Param((1,n))

In [4]:
r_node = linear_node(X_node,w_node)
z_node = bias_node(r_node,b_node)
# J_node = softmax_mce_node(z_node,y_node)
h_node = softmax_node(z_node)
J_node = mce_node(h_node,y_node)

In [5]:
g = Graph()
g.build(J_node).initialize().feed({X_node:predictors, y_node:responses})

In [6]:
# %%timeit
g.forward().backward()

In [7]:
def softmax(elements):
    shift = elements.max(axis=1).reshape(-1,1)
    exp = np.exp(elements - shift)
    return exp / exp.sum(axis=1).reshape(-1,1)

def compute_cost(w,X,y):
    m,d = X.shape
    W = w.reshape((d,-1))
    prob = softmax(X.dot(W))
    loglikelihood = np.log(prob[range(m),y.flatten()]).sum() 
    return -loglikelihood

def compute_grad(w,X,y):
    # Reshape the weights into d,n
    m,d = X.shape
    W = w.reshape((d,-1))
    _,n = W.shape
    # Evaluates the indicator function for y (one-hot-encoding)
    indicator = np.zeros((m,n))
    indicator[range(len(y)),y.flatten()] = 1
    prob = softmax(X.dot(W))
    # Builds gradient
    diff = indicator - prob
    grad = np.zeros((d,n))
    for c in range(n):
        grad[:,c] = -(X * diff[:,c].reshape((m,1))).sum(axis=0)
    grad = grad.flatten()
    return grad

In [8]:
X = np.hstack((np.ones((m,1)), predictors))
w = np.vstack((b_node.value,w_node.value))
y = y_node.value

In [9]:
print('Convetional:', compute_cost(w,X,y))
print('Graph:', J_node.value)

Convetional: 255.47998200002803
Graph: 255.47998200002803


In [10]:
print('Conventional:', compute_grad(w,X,y).flatten())

Conventional: [ -37.29247915  -25.85918224   63.15166139 -174.87365033 -146.50164898
  321.37529931 -133.40061661  -70.5852348   203.98585141  -20.31994661
  -93.68251643  114.00246304    5.22547629  -26.61624693   21.39077063]


In [11]:
print('Graph:', np.hstack((b_node.gradient.flatten(), w_node.gradient.flatten())))

Graph: [ -37.29247915  -25.85918224   63.15166139 -174.87365033 -146.50164898
  321.37529931 -133.40061661  -70.5852348   203.98585141  -20.31994661
  -93.68251643  114.00246304    5.22547629  -26.61624693   21.39077063]
